# U.S. Population Migration Data:
### Strengths and Limitations

#### <center><b><U> Census Bureau Processing </U> </b></center>

   The first step in creating the migration data file is to assign a geographic code (geocode) to the IMF data.  The Census assigns these geocodes based on “ZIP plus 4” codes and State of residence reported on the tax return.  The “plus 4” codes actually consist of a pair of two-character codes—a sector code and a segment code.  According to U.S. Post Office guidelines, each sector code identifies a single county.  Using the combination of ZIP sector codes and State of residence codes for each individual return, Census assigns each record a State/county geocode.  To prepare the migration data, which examine year-to-year changes, Census must geocode 2 consecutive filing years of IMF data.  County equivalent codes are assigned to the District of Columbia, the Virgin Islands, Puerto Rico, APO/FPO (military), and “other foreign” areas.
	
#### <center><b><U> Identifying Migrants </U> </b></center>

   Once the geographic codes are assigned, Census determines who in the file has, or has not, migrated.  To do this; 
  -  first, coded returns for the current filing year are matched to coded returns filed during the prior year. 
   
  - The mailing addresses on the two returns are then compared to one another focusing on:  (1) the street address and (2) State plus ZIP code. 
   
  - <b>If the two are identical, the return is labeled a <font color='red'>“non-migrant.”</font></b>
  
  -If any of the above information changed between the 2 years, the return is considered a mover.  However, <b>the return is only classified a <font color='red'>“migrant” </font> if the taxpayer’s geographic code also changed from one year to the next.</b>
  
  - For cases in which the <b> geographic code did change from one year to the next</b>, a taxpayer is considered an <b><font color='red'>“in-migrant” </font></b> for the address on the return filed in the current filing year, and an <font color='red'><b> “out-migrant” </b></font> for the address on the return filed for the prior year.  

#### <center><b><U> Limitations and Margin of Error in dataset:</U> </b></center>

   Although the filer’s return address determines the migration status of the record, there are instances for which the taxpayer may not have changed residences but the return address suggests a move.  This may happen if:  
   1.  the filing address is that of a financial institution or tax preparer, and not that of the actual taxpayer; 
   2.  the taxpayer is a college student living away from home who filed with a home address one year and the college address another; 
   3.  the taxpayer reports his or her place of business as the return address; 
   4.  the taxpayer maintains dual residences, primarily residing in one county but filing the tax return from the other; or 
   5.  the taxpayer uses a post office box for mailing purposes.

As mentioned, those who are not required to file United States Federal income tax returns are not included in this file, and so <b>the data under-represent the poor and the elderly</b>.  Also excluded is the small percentage of tax returns filed after late September of the filing year.  Most taxpayers whose returns are filed after this date have been granted an extension to file by the IRS.  These taxpayers are likely to have complex returns that report relatively high income, and so the migration data set may under-represent the very wealthy, as well.

   The matching process also causes some returns to be excluded from the counts.  <b>When the current-year tax return is compared to the prior-year tax return, only the Social Security Number of the primary taxpayer is considered.</b>  If a secondary filer exists (as in the case of a married couple filing jointly), that Social Security Number is not recorded or compared in creating the migration dataset.  If, for example, a husband and wife file a joint return in the prior year, but divorce and file separately in the current year, only the husband’s current-year return will have a match with the prior-year return.  The now ex-wife’s current-year return becomes a non-match and will not be included in the data counts.  Other changes in filing status—from from joint to married filing separately—will also affect the data.



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import glob as glob
import pandas as pd
!pip install xlrd --upgrade
!pip install patool

In [ ]:
!pip install patool
import patoolib

# ETL Inflow Files

Unzip rar file 



1.   Read invdividual files for all states
2.   Process and Clean 
3.   Merge All





In [151]:
def  read_xls(path_xls):
  """read xls file, remove header rows, rename columns"""
  df = pd.read_excel(path_xls, engine='xlrd',skiprows=8, header=None , names=['destination_state', 'destination_county', 'origin_state','origin_county',
                                                                     'origin_usps','county_names', 'numb_returns', 'numb_exemptions','agi'])
  return df


def set_fips(df):
  """Calculate county fips using state and county area code """
  cleaned_df = pd.DataFrame()
  cleaned_df['origin_countyfips'] = df.origin_state*1000 + df.origin_county
  cleaned_df['destination_countyfips'] = df.destination_state*1000 + df.destination_county
  cleaned_df['county'] = df.county_names
  cleaned_df['returns'] = df.numb_returns
  cleaned_df['exemptions'] = df.numb_exemptions
  return cleaned_df


def inflow_state(cleaned_df, format=1):
  """Merge migrant and non migrant datasets to calculate %Inflow."""

  if format==1:
    migrant = cleaned_df[cleaned_df.origin_countyfips == 1].reset_index(drop=True)
    migrant.rename(columns={'returns':'inmigrant_returns', 'exemptions':'inmigrant_exemptions'}, inplace=True)
    migrant.drop(columns=['origin_countyfips'], inplace=True)

    nonmigrant = cleaned_df[cleaned_df.origin_countyfips==63050].reset_index(drop=True)
    nonmigrant.rename(columns={'returns':'nonmigrant_returns', 'exemptions':'nonmigrant_exemptions'}, inplace=True)
    nonmigrant.drop(columns=['origin_countyfips', 'county'], inplace=True)
  
  elif format==2:
    migrant = cleaned_df[cleaned_df.origin_countyfips==96000].reset_index(drop=True).drop(columns='origin_countyfips')
    migrant.rename(columns={'returns':'inmigrant_returns', 'exemptions':'inmigrant_exemptions'}, inplace=True)
    migrant = migrant[~(migrant.county=='Total Mig - US & For')].reset_index(drop=True)

    nonmigrant = cleaned_df[cleaned_df.destination_countyfips==cleaned_df.origin_countyfips].reset_index(drop=True).drop(columns=['origin_countyfips','county'])
    nonmigrant.rename(columns={'returns':'nonmigrant_returns', 'exemptions':'nonmigrant_exemptions'}, inplace=True)

  inflow = migrant.merge(nonmigrant, on='destination_countyfips', how='left')
  inflow = inflow[~(inflow.inmigrant_returns=='d') | ~(inflow.nonmigrant_returns=='d')].reset_index(drop=True)
  inflow.inmigrant_returns = inflow.inmigrant_returns.astype('str').str.extract('(\d+)').astype('float')
  inflow.nonmigrant_returns = inflow.nonmigrant_returns.astype('str').str.extract('(\d+)').astype('float')
  inflow['%inflow'] = 100*(inflow['inmigrant_returns']/inflow['nonmigrant_returns'])
  return inflow


def process_year(xls_files, format=1):

  """Process and merge all states for a given Year"""
  inflow_allstates = pd.DataFrame()
  for xls_file in xls_files:
    df = read_xls(xls_file)
    cleaned_df = set_fips(df)
    inflow_df = inflow_state(cleaned_df, format)
    inflow_allstates = inflow_allstates.append(inflow_df)
  return inflow_allstates

## Format 1 parsing from Year 1993-2005

In [ ]:
path_raw = "/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw"
zip_files_path = glob.glob(path_raw + "/*.zip")
years = list(range(1993,2005))
unzip_path = "/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped/"

for zippath in zip_files_path:
  patoolib.extract_archive(zippath,outdir=unzip_path)

In [ ]:
unzipped_folders = glob.glob("/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped/*/*Inflow")

In [ ]:
unzipped_folders9295 = unzipped_folders[0:3]
years = [1993,1994,1995]
i=0
inflow_9295 = pd.DataFrame()
for unzipped_folder in unzipped_folders9295:
  xls_files_i = glob.glob(unzipped_folder + "/*.xls")
  inflow_i = process_year(xls_files_i)
  inflow_i['Year'] = years[i]
  inflow_9295 = inflow_9295.append(inflow_i)
  i = i+1
inflow_9295 = inflow_9295.reset_index(drop=True)
inflow_9295 = inflow_9295[~(inflow_9295['nonmigrant_exemptions'].isna())].reset_index(drop=True)
inflow_9295.inmigrant_exemptions = inflow_9295.inmigrant_exemptions.astype('float64')
inflow_9295.nonmigrant_exemptions = inflow_9295.nonmigrant_exemptions.astype('float64')

In [ ]:
inflow_9295.append(inflow_on95).reset_index(drop=True).to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_inflow1993_2004.csv', index=False)

In [31]:
inflow_9295 = pd.read_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_inflow1993_2004.csv')

In [34]:
#@title Annual Migration Inflow returns filed, 1993-1995
df = inflow_9295.copy()
df['County Code'] = df.destination_countyfips.astype('str').str.zfill(5)
fig = px.choropleth(df, locations="County Code",
                        geojson=counties,
                        color="%inflow",
                        hover_name="county",
                        scope="usa",
                        animation_frame='Year',
                        range_color=(0, 30),
                        color_continuous_scale="ylorrd")
    
fig.update_layout(showlegend=False, title='Annual Migration Inflow returns filed, 1993-1995')
fig.update_traces(marker_line_width=0.25)
fig.show()

## Format 2 parsing from Year 1996-2004

In [ ]:
unzipped_folder95onw = unzipped_folders[3:]
years = [1998,1997,1996,2000,1999,2003,2002,2001,2004]
i=0
inflow_on95 = pd.DataFrame()
for unzipped_folder in unzipped_folder95onw:
  xls_files_i = glob.glob(unzipped_folder + "/*.xls")
  inflow_i = process_year(xls_files_i, format=2)
  inflow_i['Year'] = years[i]
  inflow_on95 = inflow_on95.append(inflow_i)
  i = i+1
inflow_on95 = inflow_on95.reset_index(drop=True)
inflow_on95 = inflow_on95[~(inflow_on95['nonmigrant_exemptions'].isna())].reset_index(drop=True)
inflow_on95.inmigrant_exemptions = inflow_on95.inmigrant_exemptions.astype('float64')
inflow_on95.nonmigrant_exemptions = inflow_on95.nonmigrant_exemptions.astype('float64')

In [ ]:
inflow_9304 = inflow_9295.append(inflow_on95).reset_index(drop=True)

In [ ]:
inflow_on95.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28170 entries, 0 to 28169
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   destination_countyfips  28170 non-null  int64  
 1   county                  28170 non-null  object 
 2   inmigrant_returns       28170 non-null  float64
 3   inmigrant_exemptions    28170 non-null  object 
 4   nonmigrant_returns      28168 non-null  float64
 5   nonmigrant_exemptions   28168 non-null  object 
 6   %inflow                 28168 non-null  float64
 7   Year                    28170 non-null  int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 1.7+ MB


## Format 3 Dat Files 2005-2011

In [ ]:
path_raw = "/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw"
zip_files_path = glob.glob(path_raw + "/county*.zip")
years = list(range(2005,2012))
unzip_path = "/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/"

for zippath in zip_files_path:
  patoolib.extract_archive(zippath,outdir=unzip_path)
unzipped_folders = glob.glob("/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/countyin*.dat")
unzipped_folders

In [ ]:
years = [2005,2006,2007,2010,2009,2011]
i=0
inflow_2004on = pd.DataFrame()
for unzip_dat in unzipped_folders:
  dat_data = pd.read_fwf(unzip_dat, header=None,encoding = "ISO-8859-1", names=['destination_state', 'destination_county', 'origin_state','origin_county',
                                                                     'origin_usps','county_names', 'numb_returns', 'numb_exemptions','agi', 'unknown'])
  dat_data.origin_countyfips = dat_data.origin_state*1000 + dat_data.origin_county
  dat_data.destination_countyfips = dat_data.destination_state*1000 + dat_data.destination_county
  cleaned_df = set_fips(dat_data)
  process_df = inflow_state(cleaned_df, format=2)
  process_df['Year'] = years[i]
  i=i+1
  inflow_2004on = inflow_2004on.append(process_df)
inflow_2004on = inflow_2004on.reset_index(drop=True)
inflow_2004on.nunique()

In [ ]:
inflow93_2011 = inflow_9304.append(inflow_2004on).reset_index(drop=True)

In [ ]:
inflow93_2011.describe()

,destination_countyfips,inmigrant_returns,inmigrant_exemptions,nonmigrant_returns,nonmigrant_exemptions,%inflow,Year
count,56426.000000,56426.000000,56426.000000,5.642600e+04,5.642600e+04,56426.000000,56426.000000
mean,30403.395509,2215.937103,4206.826764,3.032793e+04,6.679927e+04,7.903940,2001.674175
std,15162.050856,5761.534846,10322.898247,9.692266e+04,2.119741e+05,4.383086,5.439342
min,1001.000000,1.000000,-1.000000,1.000000e+00,-1.000000e+00,1.780415,1993.000000
25%,18179.000000,244.000000,518.000000,3.403000e+03,7.934000e+03,5.722452,1997.000000
50%,29175.000000,563.000000,1166.000000,7.765000e+03,1.790950e+04,7.096774,2002.000000
75%,45083.000000,1543.000000,3082.000000,2.036775e+04,4.606325e+04,9.045557,2006.000000
max,57007.000000,110030.000000,193646.000000,3.290970e+06,7.244412e+06,100.000000,2011.000000


In [ ]:
inflow93_2011.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/inflow1993_2011.csv', index=False)

## Format 4 - Latest 2012-2020

In [ ]:
path_raw = "/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw"
zip_files_path = glob.glob(path_raw + "/*migrationdata.zip")

In [ ]:
unzip_path = "/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/"

for zippath in zip_files_path:
  patoolib.extract_archive(zippath,outdir=unzip_path)

In [ ]:
unzipped_folders = glob.glob("/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyinflow*.csv")
unzipped_folders

['/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyinflow1112.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyinflow1314.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyinflow1213.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyinflow1617.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyinflow1516.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/cou

In [ ]:
years = [2012,2014,2013,2017,2016,2015,2019,2018,2020]
i=0
inflow_1221 = pd.DataFrame()
for csv_path in unzipped_folders:

  df = pd.read_csv(csv_path, encoding = "ISO-8859-1")
  cleaned_df = pd.DataFrame()
  cleaned_df['origin_countyfips'] = df.y1_statefips*1000 + df.y1_countyfips
  cleaned_df['destination_countyfips'] = df.y2_statefips*1000 + df.y2_countyfips
  cleaned_df['county'] = df.y1_countyname
  cleaned_df['returns'] = df.n1
  cleaned_df['individuals'] = df.n2
  migrant = cleaned_df[cleaned_df.origin_countyfips==96000].reset_index(drop=True).drop(columns='origin_countyfips')
  migrant.rename(columns={'returns':'inmigrant_returns', 'individuals':'inmigrants'}, inplace=True)
  migrant = migrant[~(migrant.county == 'Total Migration-US and Foreign')].reset_index(drop=True)
  nonmigrant = cleaned_df[cleaned_df.destination_countyfips==cleaned_df.origin_countyfips].reset_index(drop=True).drop(columns=['origin_countyfips','county'])
  nonmigrant.rename(columns={'returns':'nonmigrant_returns', 'individuals':'nonmigrants'}, inplace=True)
  inflow = migrant.merge(nonmigrant, on='destination_countyfips', how='left')
  inflow = inflow[~(inflow.inmigrant_returns=='d') | ~(inflow.nonmigrant_returns=='d')].reset_index(drop=True)
  inflow.inmigrant_returns = inflow.inmigrant_returns.astype('str').str.extract('(\d+)').astype('float')
  inflow.nonmigrant_returns = inflow.nonmigrant_returns.astype('str').str.extract('(\d+)').astype('float')
  inflow['%inflow'] = 100*(inflow['inmigrant_returns']/inflow['nonmigrant_returns'])
  inflow['Year'] = years[i]
  inflow_1221 = inflow_1221.append(inflow)
  i=i+1

inflow_1221.reset_index(drop=True)



,destination_countyfips,county,inmigrant_returns,inmigrants,nonmigrant_returns,nonmigrants,%inflow,Year
0,1001,Autauga County Total Migration-US and Foreign,2006.0,4618,17696.0,41757.0,11.335895,2012
1,1003,Baldwin County Total Migration-US and Foreign,5633.0,11518,65215.0,144258.0,8.637583,2012
2,1005,Barbour County Total Migration-US and Foreign,665.0,1440,8001.0,17972.0,8.311461,2012
3,1007,Bibb County Total Migration-US and Foreign,479.0,1062,6198.0,14840.0,7.728299,2012
4,1009,Blount County Total Migration-US and Foreign,1257.0,2767,17390.0,42898.0,7.228292,2012
...,...,...,...,...,...,...,...,...
28172,56037,Sweetwater County Total Migration-US and Foreign,833.0,1673,15424.0,35193.0,5.400674,2020
28173,56039,Teton County Total Migration-US and Foreign,1275.0,1981,10840.0,20640.0,11.761993,2020
28174,56041,Uinta County Total Migration-US and Foreign,475.0,1083,7102.0,17922.0,6.688257,2020
28175,56043,Washakie County Total Migration-US and Foreign,205.0,396,2885.0,6592.0,7.105719,2020


In [ ]:
inflow_all = inflow93_2011.drop(columns=['inmigrant_exemptions', 'nonmigrant_exemptions']).append(inflow_1221.drop(columns=['inmigrants', 'nonmigrants']))
inflow_all
inflow_all.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/inflow_all.csv', index=False)
inflow_1221.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/inflow_1221.csv', index=False)

,destination_countyfips,county,inmigrant_returns,nonmigrant_returns,%inflow,Year
0,57001,Foreign / Overseas,35338.0,151429.0,23.336349,1993
1,57003,Foreign / Puerto Rico,6691.0,67648.0,9.890906,1993
2,57005,APO / FPO Zip Code,94040.0,169479.0,55.487701,1993
3,57007,Foreign / Virgin Islands,3211.0,5102.0,62.936103,1993
4,56001,Albany,1561.0,9384.0,16.634697,1993
...,...,...,...,...,...,...
3097,56037,Sweetwater County Total Migration-US and Foreign,833.0,15424.0,5.400674,2020
3098,56039,Teton County Total Migration-US and Foreign,1275.0,10840.0,11.761993,2020
3099,56041,Uinta County Total Migration-US and Foreign,475.0,7102.0,6.688257,2020
3100,56043,Washakie County Total Migration-US and Foreign,205.0,2885.0,7.105719,2020


In [155]:
inflow_all = pd.read_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/inflow_all.csv')
inflow_all

,destination_countyfips,county,inmigrant_returns,nonmigrant_returns,%inflow,Year
0,57001,Foreign / Overseas,35338.0,151429.0,23.336349,1993
1,57003,Foreign / Puerto Rico,6691.0,67648.0,9.890906,1993
2,57005,APO / FPO Zip Code,94040.0,169479.0,55.487701,1993
3,57007,Foreign / Virgin Islands,3211.0,5102.0,62.936103,1993
4,56001,Albany,1561.0,9384.0,16.634697,1993
...,...,...,...,...,...,...
84598,56037,Sweetwater County Total Migration-US and Foreign,833.0,15424.0,5.400674,2020
84599,56039,Teton County Total Migration-US and Foreign,1275.0,10840.0,11.761993,2020
84600,56041,Uinta County Total Migration-US and Foreign,475.0,7102.0,6.688257,2020
84601,56043,Washakie County Total Migration-US and Foreign,205.0,2885.0,7.105719,2020


In [152]:
#@title Append data from 2008 dat file and remove outlier

inflow_all = pd.read_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/inflow_all.csv')

inflow_2008 = pd.DataFrame()
unzip_dat = '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/ci0708us.dat'
dat_data = pd.read_fwf(unzip_dat, header=None,encoding = "ISO-8859-1", names=['destination_state', 'destination_county', 'origin_state','origin_county',
                                                                   'origin_usps','county_names', 'numb_returns', 'numb_exemptions','agi', 'unknown'])
dat_data.origin_countyfips = dat_data.origin_state*1000 + dat_data.origin_county
dat_data.destination_countyfips = dat_data.destination_state*1000 + dat_data.destination_county
cleaned_df = set_fips(dat_data)
process_df = inflow_state(cleaned_df, format=2)
process_df['Year'] = 2008
i=i+1
inflow_2008 = inflow_2008.append(process_df)
inflow_2008 = inflow_2008.reset_index(drop=True)
inflow_2008.nunique()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



destination_countyfips    3142
county                    1875
inmigrant_returns         1805
inmigrant_exemptions      2230
nonmigrant_returns        2976
nonmigrant_exemptions     3056
%inflow                   3128
Year                         1
dtype: int64

In [156]:
inflow_all = inflow_all.append(inflow_2008.drop(columns=['inmigrant_exemptions','nonmigrant_exemptions'])).reset_index(drop=True)
inflow_all = inflow_all.sort_values(by='Year').reset_index(drop=True)

In [165]:
inflow_all[(inflow_all.inmigrant_returns==1)]

,destination_countyfips,county,inmigrant_returns,nonmigrant_returns,%inflow,Year
37759,38087,Slope County Tot Mig-US & For,1.0,1.0,0.0,2005
38110,30103,Treasure County Tot Mig-US & Fo,1.0,1.0,0.0,2005
38117,31005,Arthur County Tot Mig-US & For,1.0,1.0,0.0,2005
38120,31009,Blaine County Tot Mig-US & For,1.0,1.0,0.0,2005
38142,30069,Petroleum Count Tot Mig-US & Fo,1.0,1.0,0.0,2005
...,...,...,...,...,...,...
80737,8017,Cheyenne County Total Migration-US and Foreign,1.0,635.0,0.0,2018
80755,8053,Hinsdale County Total Migration-US and Foreign,1.0,291.0,0.0,2018
81061,28055,Issaquena County Total Migration-US and Foreign,1.0,324.0,0.0,2018
81184,20071,Greeley County Total Migration-US and Foreign,1.0,482.0,0.0,2018


In [164]:
inflow_all.loc[(inflow_all.inmigrant_returns==1),'%inflow'] = 0

In [166]:
inflow_all.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/inflow_all_final.csv', index=False)


In [159]:
inflow_all.nunique()

destination_countyfips     3158
county                     8252
inmigrant_returns         11833
nonmigrant_returns        37619
%inflow                   82496
Year                         28
dtype: int64

In [160]:
28*3158

88424

In [157]:
inflow_all

,destination_countyfips,county,inmigrant_returns,nonmigrant_returns,%inflow,Year
0,57001,Foreign / Overseas,35338.0,151429.0,23.336349,1993
1,21117,Kenton,4540.0,48791.0,9.304995,1993
2,21119,Knott,172.0,3649.0,4.713620,1993
3,21121,Knox,419.0,6766.0,6.192728,1993
4,21123,Larue,259.0,3688.0,7.022777,1993
...,...,...,...,...,...,...
87740,40029,Coal County Total Migration-US and Foreign,144.0,1531.0,9.405617,2020
87741,40027,Cleveland County Total Migration-US and Foreign,9779.0,93352.0,10.475405,2020
87742,40023,Choctaw County Total Migration-US and Foreign,333.0,4219.0,7.892866,2020
87743,39147,Seneca County Total Migration-US and Foreign,1181.0,21161.0,5.581022,2020


# ETL Outtflow Files

In [9]:
def  read_xls_outflow(path_xls):
  """read xls file, remove header rows, rename columns"""
  df = pd.read_excel(path_xls, engine='xlrd',skiprows=8, header=None , names=['origin_state','origin_county','destination_state', 'destination_county', 
                                                                     'destination_usps','county_names', 'numb_returns', 'numb_exemptions','agi'])
  return df


def set_fips(df):
  """Calculate county fips using state and county area code """
  cleaned_df = pd.DataFrame()
  cleaned_df['origin_countyfips'] = df.origin_state*1000 + df.origin_county
  cleaned_df['destination_countyfips'] = df.destination_state*1000 + df.destination_county
  cleaned_df['county'] = df.county_names
  cleaned_df['returns'] = df.numb_returns
  cleaned_df['exemptions'] = df.numb_exemptions
  return cleaned_df


def outflow_state(cleaned_df, format=1):
  """Merge migrant and non migrant datasets to calculate %outflow."""

  if format==1:
    migrant = cleaned_df[cleaned_df.destination_countyfips == 1].reset_index(drop=True)
    migrant.rename(columns={'returns':'outmigrant_returns', 'exemptions':'outmigrant_exemptions'}, inplace=True)
    migrant.drop(columns=['destination_countyfips'], inplace=True)

    nonmigrant = cleaned_df[cleaned_df.destination_countyfips==63050].reset_index(drop=True)
    nonmigrant.rename(columns={'returns':'nonmigrant_returns', 'exemptions':'nonmigrant_exemptions'}, inplace=True)
    nonmigrant.drop(columns=['destination_countyfips', 'county'], inplace=True)
  
  elif format==2:
    migrant = cleaned_df[cleaned_df.destination_countyfips==96000].reset_index(drop=True).drop(columns='destination_countyfips')
    migrant.rename(columns={'returns':'outmigrant_returns', 'exemptions':'outmigrant_exemptions'}, inplace=True)
    migrant = migrant[~(migrant.county=='Total Mig - US & For')].reset_index(drop=True)

    nonmigrant = cleaned_df[cleaned_df.destination_countyfips==cleaned_df.origin_countyfips].reset_index(drop=True).drop(columns=['destination_countyfips','county'])
    nonmigrant.rename(columns={'returns':'nonmigrant_returns', 'exemptions':'nonmigrant_exemptions'}, inplace=True)

  outflow = migrant.merge(nonmigrant, on='origin_countyfips', how='left')
  outflow = outflow[~(outflow.outmigrant_returns=='d') | ~(outflow.nonmigrant_returns=='d')].reset_index(drop=True)
  outflow.outmigrant_returns = outflow.outmigrant_returns.astype('str').str.extract('(\d+)').astype('float')
  outflow.nonmigrant_returns = outflow.nonmigrant_returns.astype('str').str.extract('(\d+)').astype('float')
  outflow['%outflow'] = 100*(outflow['outmigrant_returns']/outflow['nonmigrant_returns'])
  return outflow


def process_year_outflow(xls_files, format=1):

  """Process and merge all states for a given Year"""
  outflow_allstates = pd.DataFrame()
  for xls_file in xls_files:
    df = read_xls_outflow(xls_file)
    cleaned_df = set_fips(df)
    outflow_df = outflow_state(cleaned_df, format)
    outflow_allstates = outflow_allstates.append(outflow_df)
  return outflow_allstates

In [101]:
import glob as glob

In [102]:
unzipped_folders = glob.glob("/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped/*/*Outflow")
unzipped_folders = unzipped_folders[1:]

## Format 1 - (Year 1993-1995)

In [ ]:
unzipped_folders9295 = unzipped_folders[0:3]
years = [1993,1994,1995]
i=0
outflow_9295 = pd.DataFrame()
for unzipped_folder in unzipped_folders9295:
  xls_files_i = glob.glob(unzipped_folder + "/*.xls")
  outflow_i = process_year_outflow(xls_files_i, format=1)
  outflow_i['Year'] = years[i]
  outflow_9295 = outflow_9295.append(outflow_i)
  i = i+1
outflow_9295 = outflow_9295.reset_index(drop=True)
outflow_9295 = outflow_9295[~(outflow_9295['nonmigrant_exemptions'].isna())].reset_index(drop=True)


In [11]:
outflow_9295.outmigrant_exemptions = outflow_9295.outmigrant_exemptions.astype('float64')
outflow_9295.nonmigrant_exemptions = outflow_9295.nonmigrant_exemptions.astype('float64')

In [49]:
outflow_9295.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_inflow1993_1995.csv', index=False)

In [48]:
outflow_9295

,origin_countyfips,county,outmigrant_returns,outmigrant_exemptions,nonmigrant_returns,nonmigrant_exemptions,%outflow,Year
0,57001,Foreign / Overseas,94415.0,125389.0,151429.0,310294.0,62.349352,1993
1,57003,Foreign / Puerto Rico,2556.0,6699.0,67648.0,124186.0,3.778382,1993
2,57005,APO / FPO Zip Code,141212.0,316363.0,169479.0,411763.0,83.321237,1993
3,57007,Foreign / Virgin Islands,1867.0,5297.0,5102.0,9542.0,36.593493,1993
4,56001,Albany,1437.0,2598.0,9384.0,20095.0,15.313299,1993
...,...,...,...,...,...,...,...,...
9398,1125,Tuscaloosa,2962.0,5519.0,46687.0,106604.0,6.344379,1995
9399,1127,Walker,1085.0,2329.0,21419.0,50819.0,5.065596,1995
9400,1129,Washington,264.0,624.0,5051.0,12800.0,5.226688,1995
9401,1131,Wilcox,218.0,499.0,3566.0,9638.0,6.113292,1995


In [13]:
outflow_9295.describe()

,origin_countyfips,outmigrant_returns,outmigrant_exemptions,nonmigrant_returns,nonmigrant_exemptions,%outflow,Year
count,9403.000000,9403.000000,9403.000000,9.403000e+03,9.403000e+03,9403.000000,9403.000000
mean,30442.507072,2093.558332,4135.697437,2.734336e+04,6.130037e+04,7.951433,1994.000000
std,15174.830960,6420.878924,12629.277136,8.812076e+04,1.949182e+05,3.861222,0.816497
min,1001.000000,13.000000,19.000000,1.380000e+02,3.210000e+02,2.901948,1993.000000
25%,18183.000000,236.000000,500.000000,3.192000e+03,7.650000e+03,6.122449,1993.000000
50%,29177.000000,520.000000,1063.000000,7.157000e+03,1.708100e+04,7.251908,1994.000000
75%,45090.000000,1347.500000,2709.000000,1.822100e+04,4.273300e+04,8.829932,1995.000000
max,57007.000000,160038.000000,334401.000000,2.782437e+06,6.437122e+06,102.862099,1995.000000


In [14]:
outflow_9295.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9403 entries, 0 to 9402
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   origin_countyfips      9403 non-null   int64  
 1   county                 9401 non-null   object 
 2   outmigrant_returns     9403 non-null   float64
 3   outmigrant_exemptions  9403 non-null   float64
 4   nonmigrant_returns     9403 non-null   float64
 5   nonmigrant_exemptions  9403 non-null   float64
 6   %outflow               9403 non-null   float64
 7   Year                   9403 non-null   int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 587.8+ KB


In [15]:
outflow_9295.nunique()

origin_countyfips        3141
county                   3680
outmigrant_returns       3212
outmigrant_exemptions    4332
nonmigrant_returns       7790
nonmigrant_exemptions    8635
%outflow                 9289
Year                        3
dtype: int64

In [22]:
import plotly.express as px
import matplotlib.pyplot as plt
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
  counties = json.load(response)

In [28]:
#@title Annual Migration outflow returns filed, 1993-1995
df = outflow_9295.copy()
df['County Code'] = df.origin_countyfips.astype('str').str.zfill(5)
fig = px.choropleth(df, locations="County Code",
                        geojson=counties,
                        color="%outflow",
                        hover_name="county",
                        scope="usa",
                        animation_frame='Year',
                        range_color=(0, 30),
                        color_continuous_scale="ylorrd")
    
fig.update_layout(showlegend=False, title='Annual Migration outflow returns filed, 1993-1995')
fig.update_traces(marker_line_width=0.25)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [35]:
outflow_9295[outflow_9295.origin_countyfips==22115]

,origin_countyfips,county,outmigrant_returns,outmigrant_exemptions,nonmigrant_returns,nonmigrant_exemptions,%outflow,Year
2019,22115,Vernon,7697.0,19748.0,12997.0,33497.0,59.221359,1993
5154,22115,Vernon (Total Migrant),5589.0,14288.0,12093.0,30866.0,46.216820,1994
8287,22115,Vernon,3921.0,9440.0,13946.0,35785.0,28.115589,1995


In [36]:
inflow_9295[inflow_9295.destination_countyfips==22115]

,destination_countyfips,county,inmigrant_returns,inmigrant_exemptions,nonmigrant_returns,nonmigrant_exemptions,%inflow,Year
2019,22115,Vernon,4428.0,10247.0,12997.0,33497.0,34.069401,1993
4303,22115,Vernon (Total Migrant),5537.0,13652.0,12093.0,30866.0,45.786819,1994
7436,22115,Vernon,3148.0,7407.0,13946.0,35785.0,22.572781,1995
10568,22115,Vernon Parish Tot Mig-US & For,2847.0,6825.0,13742.0,34471.0,20.717508,1998
13699,22115,Vernon Parish Tot Mig-US & For,2698.0,6182.0,14039.0,35587.0,19.217893,1997
16825,22115,Vernon Parish Tot Mig-US & For,3139.0,7585.0,13336.0,33847.0,23.537792,1996
19953,22115,Vernon Parish Tot Mig-US & For,3252.0,7381.0,14278.0,34819.0,22.776299,2000
23083,22115,Vernon Parish Tot Mig-US & For,2992.0,6567.0,14077.0,34806.0,21.254529,1999
26218,22115,Vernon Parish Tot Mig-US & For,3044.0,6771.0,14236.0,34301.0,21.382411,2003
29348,22115,Vernon Parish Tot Mig-US & For,2938.0,6406.0,14417.0,34749.0,20.378720,2002


## Format 2 (Year 1996-2004)

In [110]:
unzipped_folder95onw = unzipped_folders[3:]
years = list(range(1996,2005))
i=0
outflow_on95 = pd.DataFrame()
for unzipped_folder in unzipped_folder95onw:
  xls_files_i = glob.glob(unzipped_folder + "/*.xls")
  outflow_i = process_year_outflow(xls_files_i, format=2)
  outflow_i['Year'] = years[i]
  outflow_on95 = outflow_on95.append(outflow_i)
  i = i+1
outflow_on95 = outflow_on95.reset_index(drop=True)
outflow_on95 = outflow_on95[~(outflow_on95['nonmigrant_exemptions'].isna())].reset_index(drop=True)
outflow_on95.outmigrant_exemptions = outflow_on95.outmigrant_exemptions.astype('float64')
outflow_on95.nonmigrant_exemptions = outflow_on95.nonmigrant_exemptions.astype('float64')

In [121]:
#@title 1997 XLS files to read

unzipped_folder97 = [unzipped_folders[4]]
years = [1997]
i=0
outflow_97 = pd.DataFrame()
for unzipped_folder in unzipped_folder97:
  xls_files_i = glob.glob(unzipped_folder + "/*.XLS")
  outflow_i = process_year_outflow(xls_files_i, format=2)
  outflow_i['Year'] = 1997
  outflow_97 = outflow_97.append(outflow_i)
outflow_97 = outflow_97.reset_index(drop=True)
outflow_97 = outflow_97[~(outflow_97['nonmigrant_exemptions'].isna())].reset_index(drop=True)
outflow_97.outmigrant_exemptions = outflow_97.outmigrant_exemptions.astype('float64')
outflow_97.nonmigrant_exemptions = outflow_97.nonmigrant_exemptions.astype('float64')


In [123]:
outflow_on95 = outflow_97.append(outflow_on95).reset_index(drop=True)
outflow_on95 = outflow_on95.sort_values(by='Year').reset_index(drop=True)

In [124]:
outflow_on95.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28256 entries, 0 to 28255
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   origin_countyfips      28256 non-null  int64  
 1   county                 28256 non-null  object 
 2   outmigrant_returns     28256 non-null  float64
 3   outmigrant_exemptions  28256 non-null  float64
 4   nonmigrant_returns     28256 non-null  float64
 5   nonmigrant_exemptions  28256 non-null  float64
 6   %outflow               28256 non-null  float64
 7   Year                   28256 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 1.7+ MB


In [125]:
outflow_on95.nunique()

origin_countyfips         3139
county                    2487
outmigrant_returns        6169
outmigrant_exemptions     8239
nonmigrant_returns       18221
nonmigrant_exemptions    22545
%outflow                 27346
Year                         9
dtype: int64

In [126]:
outflow_on95.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/outflow_19962004.csv', index=False)

In [127]:
outflow_922004 = outflow_9295.append(outflow_on95).reset_index(drop=True)
outflow_922004 = outflow_922004.sort_values(by='Year').reset_index(drop=True)
outflow_922004.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/outflow_19932004.csv', index=False)

## Format 3 Dat Files (Year 2005-2011)

In [128]:
unzipped_folders = glob.glob("/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/countyout*.dat")
unzipped_folders

['/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/countyout0405us1.dat',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/countyout0506.dat',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/countyout0607.dat',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/countyoutflow0809.dat',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/countyoutflow0910.dat',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/countyoutflow1011.dat']

In [130]:
years = [2005,2006,2007,2009,2010,2011]
i=0
outflow_2004on = pd.DataFrame()
for unzip_dat in unzipped_folders:
  dat_data = pd.read_fwf(unzip_dat, header=None,encoding = "ISO-8859-1", names=['origin_state','origin_county','destination_state', 'destination_county', 
                                                                     'destination_usps','county_names', 'numb_returns', 'numb_exemptions','agi', 'unknown'])
  dat_data.origin_countyfips = dat_data.origin_state*1000 + dat_data.origin_county
  dat_data.destination_countyfips = dat_data.destination_state*1000 + dat_data.destination_county
  cleaned_df = set_fips(dat_data)
  process_df = outflow_state(cleaned_df, format=2)
  process_df['Year'] = years[i]
  i=i+1
  outflow_2004on = outflow_2004on.append(process_df)
outflow_2004on = outflow_2004on.reset_index(drop=True)
outflow_2004on.nunique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



origin_countyfips         3146
county                    1935
outmigrant_returns        5092
outmigrant_exemptions     6705
nonmigrant_returns       13922
nonmigrant_exemptions    16188
%outflow                 18387
Year                         6
dtype: int64

In [132]:
3146*6

18876

In [131]:
outflow_2004on.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18856 entries, 0 to 18855
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   origin_countyfips      18856 non-null  int64  
 1   county                 18856 non-null  object 
 2   outmigrant_returns     18856 non-null  float64
 3   outmigrant_exemptions  18856 non-null  int64  
 4   nonmigrant_returns     18856 non-null  float64
 5   nonmigrant_exemptions  18856 non-null  int64  
 6   %outflow               18856 non-null  float64
 7   Year                   18856 non-null  int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 1.2+ MB


In [135]:
#@title 2008 Dat file
outflow_2008 = pd.DataFrame()
unzip_dat = '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_dat/co0708us.dat'
dat_data = pd.read_fwf(unzip_dat, header=None,encoding = "ISO-8859-1", names=['origin_state','origin_county','destination_state', 'destination_county', 
                                                                   'destination_usps','county_names', 'numb_returns', 'numb_exemptions','agi', 'unknown'])
dat_data.origin_countyfips = dat_data.origin_state*1000 + dat_data.origin_county
dat_data.destination_countyfips = dat_data.destination_state*1000 + dat_data.destination_county
cleaned_df = set_fips(dat_data)
process_df = outflow_state(cleaned_df, format=2)
process_df['Year'] = 2008
outflow_2008 = outflow_2008.append(process_df)
outflow_2008 = outflow_2008.reset_index(drop=True)
outflow_2008.nunique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



origin_countyfips        3142
county                   1875
outmigrant_returns       1834
outmigrant_exemptions    2240
nonmigrant_returns       2976
nonmigrant_exemptions    3056
%outflow                 3116
Year                        1
dtype: int64

In [136]:
outflow_2008.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   origin_countyfips      3142 non-null   int64  
 1   county                 3142 non-null   object 
 2   outmigrant_returns     3142 non-null   float64
 3   outmigrant_exemptions  3142 non-null   int64  
 4   nonmigrant_returns     3142 non-null   float64
 5   nonmigrant_exemptions  3142 non-null   int64  
 6   %outflow               3142 non-null   float64
 7   Year                   3142 non-null   int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 196.5+ KB


In [137]:
outflow_1992_2011 = outflow_922004.append(outflow_2004on).append(outflow_2008).reset_index(drop=True)
outflow_1992_2011.sort_values(by='Year').reset_index(drop=True)
outflow_1992_2011.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/outflow_1992_2011.csv', index=False)

## Format 4 (Year 2012-2020)

In [138]:
unzipped_folders = glob.glob("/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyoutflow*.csv")
unzipped_folders

['/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyoutflow1112.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyoutflow1314.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyoutflow1213.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyoutflow1415.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_2021/countyoutflow1516.csv',
 '/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/migration_raw/unzipped_2012_202

In [141]:
years = [2012,2014,2013,2015,2016,2017,2018,2019,2020]
i=0
outflow_1221 = pd.DataFrame()
for csv_path in unzipped_folders:
  df = pd.read_csv(csv_path, encoding = "ISO-8859-1")
  cleaned_df = pd.DataFrame()
  cleaned_df['origin_countyfips'] = df.y1_statefips*1000 + df.y1_countyfips
  cleaned_df['destination_countyfips'] = df.y2_statefips*1000 + df.y2_countyfips
  cleaned_df['county'] = df.y2_countyname
  cleaned_df['returns'] = df.n1
  cleaned_df['individuals'] = df.n2
  migrant = cleaned_df[cleaned_df.destination_countyfips==96000].reset_index(drop=True).drop(columns='destination_countyfips')
  migrant.rename(columns={'returns':'outmigrant_returns', 'individuals':'outmigrants'}, inplace=True)
  migrant = migrant[~(migrant.county == 'Total Migration-US and Foreign')].reset_index(drop=True)
  nonmigrant = cleaned_df[cleaned_df.destination_countyfips==cleaned_df.origin_countyfips].reset_index(drop=True).drop(columns=['destination_countyfips','county'])
  nonmigrant.rename(columns={'returns':'nonmigrant_returns', 'individuals':'nonmigrants'}, inplace=True)
  outflow = migrant.merge(nonmigrant, on='origin_countyfips', how='left')
  outflow = outflow[~(outflow.outmigrant_returns=='d') | ~(outflow.nonmigrant_returns=='d')].reset_index(drop=True)
  outflow.outmigrant_returns = outflow.outmigrant_returns.astype('str').str.extract('(\d+)').astype('float')
  outflow.nonmigrant_returns = outflow.nonmigrant_returns.astype('str').str.extract('(\d+)').astype('float')
  outflow['%outflow'] = 100*(outflow['outmigrant_returns']/outflow['nonmigrant_returns'])
  outflow['Year'] = years[i]
  outflow_1221 = outflow_1221.append(outflow)
  i=i+1

outflow_1221.reset_index(drop=True)



,origin_countyfips,county,outmigrant_returns,outmigrants,nonmigrant_returns,nonmigrants,%outflow,Year
0,1001,Autauga County Total Migration-US and Foreign,2125.0,4776,17696.0,41757.0,12.008363,2012
1,1003,Baldwin County Total Migration-US and Foreign,4327.0,8728,65215.0,144258.0,6.634977,2012
2,1005,Barbour County Total Migration-US and Foreign,771.0,1599,8001.0,17972.0,9.636295,2012
3,1007,Bibb County Total Migration-US and Foreign,523.0,1184,6198.0,14840.0,8.438206,2012
4,1009,Blount County Total Migration-US and Foreign,1252.0,2808,17390.0,42898.0,7.199540,2012
...,...,...,...,...,...,...,...,...
28193,56037,Sweetwater County Total Migration-US and Foreign,1084.0,2306,15424.0,35193.0,7.028008,2020
28194,56039,Teton County Total Migration-US and Foreign,1152.0,1867,10840.0,20640.0,10.627306,2020
28195,56041,Uinta County Total Migration-US and Foreign,476.0,1037,7102.0,17922.0,6.702337,2020
28196,56043,Washakie County Total Migration-US and Foreign,171.0,306,2885.0,6592.0,5.927210,2020


In [142]:
outflow_all = outflow_1992_2011.drop(columns=['outmigrant_exemptions', 'nonmigrant_exemptions']).append(outflow_1221.drop(columns=['outmigrants', 'nonmigrants']))
outflow_all
outflow_all.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/outflow_all.csv', index=False)
outflow_1221.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/outflow_1221.csv', index=False)

In [146]:
#@title valid return value starts from 10 as per meta file. Filtering some of the data points per that criterian
outflow_all[(outflow_all.outmigrant_returns==1)]['%outflow'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [147]:
outflow_all[outflow_all.outmigrant_returns==1]

,origin_countyfips,county,outmigrant_returns,nonmigrant_returns,%outflow,Year
38205,15005,Kalawao County Tot Mig-US & For,1.0,1.0,0.0,2005
39289,30069,Petroleum Count Tot Mig-US & Fo,1.0,1.0,0.0,2005
39306,30103,Treasure County Tot Mig-US & Fo,1.0,1.0,0.0,2005
39313,31005,Arthur County Tot Mig-US & For,1.0,1.0,0.0,2005
39315,31009,Blaine County Tot Mig-US & For,1.0,1.0,0.0,2005
...,...,...,...,...,...,...
2391,46063,Harding County Total Migration-US and Foreign,1.0,503.0,0.0,2018
2394,46069,Hyde County Total Migration-US and Foreign,1.0,518.0,0.0,2018
2652,48261,Kenedy County Total Migration-US and Foreign,1.0,111.0,0.0,2018
2656,48269,King County Total Migration-US and Foreign,1.0,89.0,0.0,2018


In [148]:
outflow_all.to_csv('/content/gdrive/MyDrive/Data/Research Data/07Research/Research Papers/Hypothesis developed from research papers/outflow_all.csv', index=False)
